In [1]:
def read_data(input_file):
    """
    Read data from stdin
    """
    with open(input_file, 'r') as f:
        lines = f.readlines()
        n, m = list(map(int, lines[0].split()))
        d = list(map(int, lines[1].split()))
        assert len(d) == n, "Number of classes does not match"
        c = list(map(int, lines[2].split()))
        assert len(c) == m, "Number of rooms does not match"
        k = int(lines[3])
        conflicts = []
        for i in range(4, 4 + k):
            conflicts.append(list(map(int, lines[i].split())))
    return n, m, d, c, conflicts


def build_conflict_table(n, conflicts):
    """
    Build a conflict table so we can check if two classes are conflicting in O(1)

    conflict_table[(i, j)] = 1 if class i and class j are conflicting
    """
    conflict_table = dict()
    for i, j in conflicts:
        conflict_table[(i, j)] = 1
        conflict_table[(j, i)] = 1

    for i in range(1, n + 1):
        conflict_table[(i, i)] = 1
        
    return conflict_table
                      

def schedule_exams(exams, num_days, num_sections, room_capacities, conflict_table):
    """
    Check whether we can schedule all exams in the given number of days
    """

    # Assign exam to a slot for each day
    num_rooms = len(room_capacities)
    schedule_exams = [[[] for _ in range(num_sections)] for _ in range(num_days)]
    schedule_rooms = [[[] for _ in range(num_sections)] for _ in range(num_days)]


    def is_valid(exam_index, day, section, room):
        """
        Check if we can schedule exam at index exam_index in day and section at room
        """
        # Check capacity
        if exams[exam_index][1] > room_capacities[room]:
            return False
        
        # Check room is not used
        if room in schedule_rooms[day][section]:
            return False
        
        # Check conflict
        for scheduled_exam in schedule_exams[day][section]:
            if conflict_table.get((exams[exam_index][0], scheduled_exam[0]), 0) == 1:
                return False
            
        return True

        

    def backtrack(exam_index):
        """
        Check if we can schedule exam at index exam_index
        """
        if exam_index == len(exams):
            return True
        
        for day in range(num_days):
            for section in range(num_sections):
                for room in range(num_rooms):
                    # Check if we can schedule exam at index exam_index in day and section
                    if is_valid(exam_index, day, section, room):
                        schedule_exams[day][section].append(exams[exam_index])
                        schedule_rooms[day][section].append(room)
                        if backtrack(exam_index + 1):
                            return True
                        schedule_exams[day][section].pop()
                        schedule_rooms[day][section].pop()

        return False
    
    if backtrack(0):
        return schedule_exams, schedule_rooms
    return None, None



class BackTrack:
    def __init__(self) -> None:
        pass    
    
    def solve(self, input_file: str):
        n, m, d, c, conflicts = read_data(input_file)
        conflict_table = build_conflict_table(n, conflicts)

        exams = [(i + 1, d[i]) for i in range(n)]

        scheduled_exams = None
        scheduled_rooms = None

        ans = 0

        for num_days in range(1, n + 1):
            scheduled_exams, scheduled_rooms = schedule_exams(exams, num_days, 4, c, conflict_table)
            if scheduled_exams is not None:
                ans = num_days
                break

        final_answer = [[] for _ in range(n)]
        for day in range(num_days):
            for section in range(4):
                for i, exam in enumerate(scheduled_exams[day][section]):
                    final_answer[exam[0] - 1] = ((exam[0], section + 1, scheduled_rooms[day][section][i] + 1))


        # # Print solution
        # print("Optimal solution:", ans)
        # print('-' * 10)
        # for i in range(n):
        #     print(f"{final_answer[i][0]} {final_answer[i][1]} {final_answer[i][2]}")

        # print('=' * 10)

        return ans, final_answer, n, m, c

In [2]:
import os
import time
import pandas as pd

test_cases = []
num_subjects = []
num_rooms = []
num_conflicts = []
times = []
objective_values = []

solver = BackTrack()

for test_case in os.listdir("../data"):
    if test_case.endswith(".txt") and (
        test_case.startswith("data_10") or
        test_case.startswith("data_16") or
        test_case.startswith("data_20") 
    ):
        print(f"Start solving {test_case}")
        start_time = time.time()
        ans, schedule, n, m, c = solver.solve("../data/" + test_case)
        end_time = time.time()
        test_cases.append(test_case)
        num_subjects.append(n)
        num_rooms.append(m)
        num_conflicts.append(len(schedule))
        times.append(end_time - start_time)
        objective_values.append(ans)
        print(f"Finished solving {test_case}")

result = pd.DataFrame({
    "test_case": test_cases,
    "num_subjects": num_subjects,
    "num_rooms": num_rooms,
    "num_conflicts": num_conflicts,
    "time": times,
    "objective_value": objective_values
})

Start solving data_10_2_(0).txt
Finished solving data_10_2_(0).txt
Start solving data_10_2_(1).txt
Finished solving data_10_2_(1).txt
Start solving data_10_2_(2).txt
Finished solving data_10_2_(2).txt
Start solving data_16_3_(0).txt
Finished solving data_16_3_(0).txt
Start solving data_16_3_(1).txt
Finished solving data_16_3_(1).txt
Start solving data_16_3_(2).txt
Finished solving data_16_3_(2).txt
Start solving data_20_4_(0).txt
Finished solving data_20_4_(0).txt
Start solving data_20_4_(1).txt
Finished solving data_20_4_(1).txt
Start solving data_20_4_(2).txt
Finished solving data_20_4_(2).txt


In [3]:
result

,test_case,num_subjects,num_rooms,num_conflicts,time,objective_value
0,data_10_2_(0).txt,10,2,10,0.005228,2
1,data_10_2_(1).txt,10,2,10,7.091517,3
2,data_10_2_(2).txt,10,2,10,0.000000,2
3,data_16_3_(0).txt,16,3,16,130.219400,2
4,data_16_3_(1).txt,16,3,16,122.805888,3
5,data_16_3_(2).txt,16,3,16,79.125592,2
6,data_20_4_(0).txt,20,4,20,0.753692,2
7,data_20_4_(1).txt,20,4,20,829.579995,2
8,data_20_4_(2).txt,20,4,20,43.922294,2


In [4]:
result.to_csv("backtrack.csv", index=False)